In [ ]:
#!pip uninstall opencv-python-headless -y
#!pip uninstall opencv-contrib-python-headless -y
#!pip install opencv-contrib-python

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None  # default='warn'
from PIL import Image
from tqdm import tqdm


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, fapt install lynxilenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Workflow
1. Preprocessing &  Plot images
2. Convert images into desired shape and size and format. for a particular `Model`
3. Create Labels of Yes or no for Images as per ID
4. Train the model.
5. Make Prediction after Train Test Split
________________________________________
* 6. Transform images!!! 
* 7. Identify contours and make bounding boxes.
* 8. feed bounding boxes to another model.
* 9. find Similarity index.
* 10. Check accuracy and precision.

____________________________

###  Details of Workflow
1. Preprocessing &  Plot images : 
`Resize, reshape and convert into grayscale and save accordingly.`

2. Use *GLCM* to identify features
3. Create Labels of Yes or no for Images as per ID
4. Train the model.
5. Make Prediction after Train Test Split

In [ ]:
fno=[]
for dirname, _, filenames in os.walk('/kaggle/input/brain-mri-images-for-brain-tumor-detection/no'):
    for filename in filenames:
        a= os.path.join(dirname, filename)
        fno.append(a)
len(fno),fno[1]        

This might be visible

In [ ]:
fyes=[]
for dirname, _, filenames in os.walk('/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes'):
    for filename in filenames:
        a= os.path.join(dirname, filename)
        fyes.append(a)
len(fyes),fyes[1]        

# Create a data Frame with File paths and labels
* 1 --> Yes (Presence of Tumor)
* 0 --> No (Absence of tumor)

In [ ]:
z = [0]* len(fno)
one = [1]*len(fyes)


In [ ]:
dfn = pd.DataFrame(list(zip(fno,z )),
               columns =['filepath', 'label'])
dfy = pd.DataFrame(list(zip(fyes,one )),
               columns =['filepath', 'label'])

df = dfn.append(dfy,ignore_index = True )
df.head()

## Creating a shuffeled Data Frame
#####dfs = df.sample(frac=1,ignore_index = True)


#######dfs.head()

# Function to Resize all the images

In [ ]:

img = Image.open(fno[2])
a = np.asarray(img)

df1 = []
for r in range (len(fno)):
    img = Image.open(fno[r])
    img= img.resize((224,224))
    a=np.asarray(img)
    df1.append(a)
    
for r in range(len(fyes)):
    img=Image.open(fyes[r])
    img=img.resize((224,224))
    a= np.asarray(img)
    df1.append(a)
    
df["Imagearr"] = df1

### Converting Image from BRG to Gray

In [ ]:
import cv2
o = df.Imagearr[3]
og = cv2.cvtColor(o,cv2.COLOR_BGR2GRAY)
Image.fromarray(o)

In [ ]:
Image.fromarray(o)

In [ ]:
# Be carefull Run only once

df["GLCM-Contrast-1"] = ''
df["GLCM-Contrast-2"] = ''
df["GLCM-Contrast-3"] = ''
df["GLCM-Contrast-4"] = ''

df["GLCM-Dissimilarity-1"] =''
df["GLCM-Dissimilarity-2"] =''
df["GLCM-Dissimilarity-3"] =''
df["GLCM-Dissimilarity-4"] =''
    
df["GLCM-Homogeneity-1"]=''
df["GLCM-Homogeneity-2"]=''
df["GLCM-Homogeneity-3"]=''
df["GLCM-Homogeneity-4"]=''
    
df["GLCM-Energy-1"]=''
df["GLCM-Energy-2"]=''
df["GLCM-Energy-3"]=''
df["GLCM-Energy-4"]=''

df["GLCM-Correlation-1"] =''
df["GLCM-Correlation-2"] =''
df["GLCM-Correlation-3"] =''
df["GLCM-Correlation-4"] =''

df["GLCM-ASM-1"]=''
df["GLCM-ASM-2"]=''
df["GLCM-ASM-3"]=''
df["GLCM-ASM-4"]=''

    

In [ ]:
df.head()

# GLCM
A statistical method of examining texture that considers the spatial relationship of pixels is the gray-level co-occurrence matrix (GLCM), also known as the gray-level spatial dependence matrix. The GLCM functions characterize the texture of an image by calculating how often pairs of pixel with specific values and in a specified spatial relationship occur in an image, creating a GLCM, and then extracting statistical measures from this matrix. (The texture filter functions, described in Calculate Statistical Measures of Texture cannot provide information about shape, that is, the spatial relationships of pixels in an image.)

After you create the GLCMs using graycomatrix, you can derive several statistics from them using graycoprops. These statistics provide information about the texture of an image.

* **greycomatrix** - calculate the grey-level co-occurrence matrix
* **greycoprops** - calculate texture properties of a GLCM

Texture properties can be 
1. **Contrast** - Measures the local variations in the gray-level co-occurrence matrix.

2. **Correlation** - Measures the joint probability occurrence of the specified pixel pairs.

3. **Energy** - Provides the sum of squared elements in the GLCM. Also known as uniformity or the angular second moment.

4. **Homogeneity** - Measures the closeness of the distribution of elements in the GLCM to the GLCM diagonal.


## Overall function to add GLCM data to the data frame.

1. function takes image arrage as an input
2. the image is converted into gray from BGR
3. this image is fed to the glcm function
4. outputs of the GLCM function are assigned to the columns of DF


In [ ]:
def fxnglcm(img,i):
    from skimage.feature import greycomatrix,greycoprops
    import skimage.feature as feature


    graycom = feature.graycomatrix(img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)


    c = feature.graycoprops(graycom, 'contrast')
    d = feature.graycoprops(graycom, 'dissimilarity')
    h = feature.graycoprops(graycom, 'homogeneity')
    e = feature.graycoprops(graycom, 'energy')
    corr = feature.graycoprops(graycom, 'correlation')
    ASM = feature.graycoprops(graycom, 'ASM')
    c = np.squeeze(c)
    d = np.squeeze(d)
    h = np.squeeze(h)
    e = np.squeeze(e)
    corr = np.squeeze(corr)
    asm = np.squeeze(ASM)
    
    df["GLCM-Contrast-1"][i] = c[0]
    df["GLCM-Contrast-2"][i] = c[1]
    df["GLCM-Contrast-3"][i] = c[2]
    df["GLCM-Contrast-4"][i] = c[3]

    df["GLCM-Dissimilarity-1"][i] =d[0]
    df["GLCM-Dissimilarity-2"][i] =d[1]
    df["GLCM-Dissimilarity-3"][i] =d[2]
    df["GLCM-Dissimilarity-4"][i] =d[3]
    
    df["GLCM-Homogeneity-1"][i] =h[0]
    df["GLCM-Homogeneity-2"][i]=h[1]
    df["GLCM-Homogeneity-3"][i] =h[2]
    df["GLCM-Homogeneity-4"][i] =h[3]
    
    df["GLCM-Energy-1"][i] =e[0]
    df["GLCM-Energy-2"][i] =e[1]
    df["GLCM-Energy-3"][i] =e[2]
    df["GLCM-Energy-4"][i] =e[3]

    df["GLCM-Correlation-1"][i] =corr[0]
    df["GLCM-Correlation-2"][i] =corr[1]
    df["GLCM-Correlation-3"][i] =corr[2]
    df["GLCM-Correlation-4"][i] =corr[3]

    df["GLCM-ASM-1"][i] =asm[0]
    df["GLCM-ASM-2"][i] =asm[1]
    df["GLCM-ASM-3"][i] =asm[2]
    df["GLCM-ASM-4"][i] =asm[3]
    
    
    
    return 


#### Overall function


In [ ]:
def glcmfiller(imgarr,i):
    import cv2
    if imgarr.shape !=(224, 224):
        og = cv2.cvtColor(imgarr,cv2.COLOR_BGR2GRAY)
        fxnglcm(og,i)
    else:
        fxnglcm(imgarr,i)


    

In [ ]:
for i in tqdm(range(253)):
    glcmfiller(df.loc[i,"Imagearr"],i)

In [ ]:
df

## Exporting Data Frame to Excel

In [ ]:
df.to_excel("./glcm.xlsx")

# Feature Detection 


### Entropy

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from skimage.filters.rank import entropy
from skimage.morphology import disk

entropy_img = entropy(og,disk(2))

fig = plt.figure(figsize = (12,12)) 
ax1 = fig.add_subplot(121)# left side
ax2 = fig.add_subplot(122)

ax1.imshow(entropy_img)
ax2.imshow(og)
plt.show()

### Gaussian Filter
Basically Blurs the image

In [ ]:
from scipy import ndimage as nd
from scipy import misc

fig = plt.figure(figsize = (12,12))
plt.gray()  # show the filtered result in grayscale
ax1 = fig.add_subplot(121) # On the Left
ax2 = fig.add_subplot(122) # Right Side
gaussian_img = nd.gaussian_filter(og,sigma = 1)
ax1.imshow(gaussian_img)
ax2.imshow(og)
plt.show()


### Sobel Filter

Helps to enhance the edges

In [ ]:
from skimage.filters import sobel

sobel_img = sobel(og)

fig = plt.figure(figsize = (12,12))
  # show the filtered result in grayscale
ax1 = fig.add_subplot(121)# left side
ax2 = fig.add_subplot(122)

ax1.imshow(sobel_img)
ax2.imshow(og)
plt.show()

### Laplace Filter

In [ ]:
from scipy.ndimage import laplace

laplace_img = laplace(og)
fig = plt.figure(figsize = (12,12))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.imshow(laplace_img)
ax2.imshow(og)
plt.show

### Gabor Filter
Gabor filter is a linear filter with a Gaussian kernel which is modulated by a sinusoidal plane wave. Frequency and orientation representations of the Gabor filter are similar to those of the human visual system. Gabor filter banks are commonly used in computer vision and image processing. They are especially suitable for edge detection and texture classification.

In [ ]:
from skimage.filters import gabor
from skimage import io

real,gabor_img = gabor(og,frequency = 0.9)

fig = plt.figure(figsize = (12,12))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.imshow(gabor_img)
ax2.imshow(og)

plt.show()


### Hessian
Filter an image with the Hybrid Hessian filter.

This filter can be used to detect continuous edges, e.g. vessels, wrinkles, rivers. It can be used to calculate the fraction of the whole image containing such objects.

In [ ]:
from skimage.filters import hessian

hessian_img = hessian(og,sigmas = range(1,100,1))
fig = plt.figure(figsize = (12,12))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.imshow(hessian_img)
ax2.imshow(og)

plt.show()

### Prewitt
Find the edge magnitude using the Prewitt transform.

In [ ]:
from skimage.filters import prewitt

prewitt_img = prewitt(og)
fig = plt.figure(figsize = (12,12))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.imshow(prewitt_img)
ax2.imshow(og)

plt.show()